# 08.3.线性回归的从零开始实现

In [1]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [2]:
def synthetic_data(w, b, num_examples):
    """生成随机样本 y = Xw + b + 噪声"""
    X = torch.normal(0,1,(num_examples,len(w)))#均值为0标准差为1 输出的张量形状是(num_examples,len(w)) 此处是1000个样本的特征，每个特征是2维向量
    y = torch.matmul(X, w) + b #torch.matmul是矩阵乘法  重申：一维向量默认是列向量  得到张量形状是(num_examples,1)
    y += torch.normal(0, 0.01, y.shape) #加上随机噪音  最终得到1000个样本的真实标签
    return X, y.reshape((-1,1)) #-1表示自动计算,1表示1列，最终把y变成了一个若干行，1列的向量

true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000) #生成1000个样本

`features`中的每一行都包含一个二维数据样本，`labels`中的每一行都包含一维标签值（一个标量）

In [3]:
print(' features:',features[0],'\n label:',labels[0])
print(' feature shape:',features.shape,'\n label shape2:',labels.shape)

 features: tensor([-2.6013, -0.2861]) 
 label: tensor([-0.0245])
 feature shape: torch.Size([1000, 2]) 
 label shape2: torch.Size([1000, 1])


In [4]:
# d2l.set_figsize()
# d2l.plt.scatter(features[:,1].detach().numpy(), labels.detach().numpy(),1);
"""
detach分离出数值，不再含有梯度
在pytorch的一些版本里面要先detach才能使用.numpy将tensor转化为numpy数组
"""

'\ndetach分离出数值，不再含有梯度\n在pytorch的一些版本里面要先detach才能使用.numpy将tensor转化为numpy数组\n'

定义一个`data_iter`函数，该函数接收批量大小、特征矩阵和标签向量作为输入，生成大小为`batch_size`的小批量

In [5]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    #这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size,num_examples)]) #每一批的大小最大为batch_indices
        yield features[batch_indices], labels[batch_indices]
    
batch_size = 10
print(features.shape)
for X, y in data_iter(batch_size, features, labels):
    print(X,'\n',y)
    break

torch.Size([1000, 2])
tensor([[ 0.3908,  0.5118],
        [-1.9959,  0.9438],
        [-0.2062,  0.4342],
        [-0.4727, -1.6695],
        [-0.0540,  0.4906],
        [ 1.4928, -0.3215],
        [ 1.7694, -0.9370],
        [ 0.4152,  0.0023],
        [ 0.4392, -1.1852],
        [-2.1318,  1.3830]]) 
 tensor([[ 3.2368],
        [-2.9979],
        [ 2.3208],
        [ 8.9372],
        [ 2.4321],
        [ 8.2909],
        [10.9120],
        [ 5.0273],
        [ 9.1098],
        [-4.7413]])


定义初始化模型参数

In [6]:
w = torch.normal(0, 0.01, size=(2,1),requires_grad = True)
b = torch.zeros(1,requires_grad = True)

定义模型

In [7]:
def linreg(X,w,b):
    """线性回归模型"""
    return torch.matmul(X,w) + b

定义损失函数

$l(\mathbf{X,y,w},b)=\frac{1}{2n} \sum_{i=1}^{n}(y_i- (\langle \mathbf{x_i}, \mathbf{w} \rangle +b))^2 = \frac{1}{2n}||\mathbf{y-(Xw}+b)||^2$

In [8]:
def squared_loss(y_hat, y):
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape))**2 / (2 * batch_size)

定义优化算法

In [9]:
def sgd(params, lr, batch_size):
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad #如果squared_loss函数里没有除以batch_size 就要在这里做
            param.grad.zero_()#清除梯度 不清除会导致梯度累加 结果不正确

训练过程

In [10]:
lr = 0.03
num_epochs = 3 #将整个数据集扫三遍
net = linreg  #返回预测的y值
loss = squared_loss
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y) #`X`和`y`的最小批损失
        # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')


epoch 1, loss 0.003798
epoch 2, loss 0.000014
epoch 3, loss 0.000005


比较真实参数和通过训练学到的参数来评估训练的成功程度

In [11]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([ 0.0008, -0.0001], grad_fn=<SubBackward0>)
b的估计误差: tensor([0.0005], grad_fn=<RsubBackward1>)


“每一次计算梯度要对整个损失函数求导，损失函数是所有样本的平均损失。这意味着每求一次梯度要把整个样本重新算一遍，计算代价大。”
求梯度要先就是对损失函数求导，示例代码中每一次求梯度时，损失函数只跟`batch_size`个样本相关，大大减小了计算量